# import

In [2]:
import cv2
import os
import os.path as osp
from glob import glob
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video

In [3]:
import time

# path

In [309]:
data_name = 'player'
input_dir = osp.join('../input', data_name)
data_dir = osp.join('../output', data_name, 'tracking')
label_save_dir = osp.join('../output', data_name, 'label')
reid_dir = osp.join('../output', data_name, 'reid')
track_label_path = osp.join('../output/')
# movie_dir = osp.join('../output', data_name, 'movie')
movie_dir = osp.join('../output', data_name, 'movie_process')
mot_pid_dir = osp.join('../output', data_name, 'mot_pid')
mot_process_save_dir = osp.join('../output', data_name, 'mot_pid_process')

# function

In [283]:
def listdirs(dir_path, name_condition=None):
    dir_names = os.listdir(dir_path)
    if name_condition is not None:
        dir_names = [dir_name for dir_name in dir_names if dir_name.startswith(name_condition)]
    dir_names.sort()
    return dir_names

In [284]:
def save_pickle(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f)


def load_pickle(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

In [285]:
def draw_bbox(img, bboxes, c=(0,255,0)):
    for bbox in bboxes:
        cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), 
                      (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])), 
                      (0,255,0), 2, lineType=cv2.LINE_AA)
        ct = [bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2]
        txt = '{}'.format(bbox[4])
        cv2.putText(img, txt, (int(ct[0]), int(ct[1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                    (0, 0, 0), thickness=3, lineType=cv2.LINE_AA)
        cv2.putText(img, txt, (int(ct[0]), int(ct[1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                    c, thickness=2, lineType=cv2.LINE_AA)

# movie_check

In [286]:
fourcc = cv2.VideoWriter_fourcc(*'H264')
columns = ['frame_index', 'tracking_id', 'xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'image_name']

# all

In [288]:
from scipy.signal import savgol_filter
def my_filter(x):
    return savgol_filter(x, 41, 1)

In [312]:
games = ['game4']
games = None

clips = ['Clip4']
clips = None

In [313]:
frame_width = 1280
frame_height = 720

In [314]:
width_ratio = 0.01
height_ratio = 0.03

In [315]:
if games is None:
    games = listdirs(input_dir, 'game')
    
columns = ['frame_index', 'tracking_id', 'xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'image_name']
for game in games:
    game_dir = osp.join(input_dir, game)
    if clips is None:
        clips = listdirs(game_dir, 'Clip')
    
    track_pid_paths = glob(osp.join(reid_dir, game, '*.csv'))
    
    for clip in clips:
        mot_pid_df_path = osp.join(mot_pid_dir, game, '{}.csv'.format(clip))
        mot_pid_df = pd.read_csv(mot_pid_df_path)
        
        person_names = mot_pid_df.person_name[~mot_pid_df.person_name.isna()].unique()
        
        mot_pid_df_list = []
        for person_name in person_names:
        
            pid_bbox_df = mot_pid_df[mot_pid_df.person_name==person_name].reset_index(drop=True)
            pid_bbox_df = pid_bbox_df.set_index('frame_index').sort_index()
            pid_bbox_df = pid_bbox_df[~pid_bbox_df.index.duplicated()]

            frame_indexes = mot_pid_df.frame_index.unique()

            frame_index_df = pd.DataFrame(frame_indexes, columns=['frame_index']).astype(int)
            frame_pid_bbox_df = frame_index_df.merge(pid_bbox_df, on=['frame_index'], how='left')

            frame_bbox_df = frame_pid_bbox_df[['frame_index', 'xmin', 'ymin', 'xmax', 'ymax']]
            frame_bbox_df = frame_bbox_df.set_index('frame_index')

            frame_bbox_df = frame_bbox_df.interpolate(method='linear', limit_direction='forward', limit_area='inside')
            frame_bbox_df = frame_bbox_df.fillna(method='bfill')
            frame_bbox_df = frame_bbox_df.fillna(method='ffill')
            frame_bbox_df = frame_bbox_df.apply(lambda x: savgol_filter(x,3,1))
            
            frame_bbox_df['xmin'] = np.maximum(frame_bbox_df.xmin * (1 - width_ratio), 0)
            frame_bbox_df['ymin'] = np.maximum(frame_bbox_df.ymin * (1 - height_ratio), 0)
            frame_bbox_df['xmax'] = np.minimum(frame_bbox_df.xmax * (1 + width_ratio), frame_width)
            frame_bbox_df['ymax'] = np.minimum(frame_bbox_df.ymax * (1 + height_ratio), frame_height)
            
            frame_bbox_df['width'] = frame_bbox_df.xmax - frame_bbox_df.xmin
            frame_bbox_df['height'] = frame_bbox_df.ymax - frame_bbox_df.ymin
            
            pid_bbox_df = frame_bbox_df.merge(pid_bbox_df[['tracking_id', 'confidence']], left_index=True, right_index=True, how='left')
            pid_bbox_df['tracking_id'] = pid_bbox_df.tracking_id.fillna(method="ffill")
            pid_bbox_df['confidence'] = pid_bbox_df.confidence.interpolate(method='linear', limit_direction='forward', limit_area='inside')
            pid_bbox_df['person_name'] = person_name
            
            pid_bbox_df[['xmin', 'ymin', 'xmax', 'ymax', 'width', 'height']] = frame_bbox_df.astype(int)
            
            mot_pid_df_list.append(pid_bbox_df)
        mot_pid_df = pd.concat(mot_pid_df_list).reset_index()
        mot_pid_process_save_path = osp.join(mot_process_save_dir, game, '{}.csv'.format(clip))
        os.makedirs(osp.dirname(mot_pid_process_save_path), exist_ok=True)
        mot_pid_df.to_csv(mot_pid_process_save_path, index=False)
        
        results = mot_pid_df[['frame_index', 'xmin', 'ymin', 'width', 'height', 'tracking_id', 'person_name']].values

        movie_save_path = osp.join(movie_dir, game, '{}.mp4'.format(clip))

        os.makedirs(osp.dirname(movie_save_path), exist_ok=True)
        out = cv2.VideoWriter(movie_save_path, fourcc, 30, (1280, 720))
        
        print('='*20, game, clip, '='*20)
        for frame_index in frame_indexes:
            bboxes = results[results[:, 0]==int(frame_index)][:, [1,2,3,4,5,6]]
            frame_path = osp.join(input_dir, game, clip,'{:04}.jpg'.format(frame_index))
            frame = cv2.imread(frame_path)
            for bbox in bboxes:
                ct = [bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2]

                person_name = bbox[5]
                if person_name is not np.nan:
                    c = (0, 0, 255)
                    cv2.putText(frame, person_name, (int(ct[0]) + 20, int(ct[1]) + 20), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                            (0, 0, 0), thickness=3, lineType=cv2.LINE_AA)
                    cv2.putText(frame, person_name, (int(ct[0]) + 20, int(ct[1]) + 20), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                            c, thickness=2, lineType=cv2.LINE_AA)
                else:
                    c = (0, 255, 0)

                cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                              (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])), 
                              c, 2, lineType=cv2.LINE_AA)

                txt = '{}'.format(bbox[4])

                cv2.putText(frame, txt, (int(ct[0]), int(ct[1])), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                            (0, 0, 0), thickness=3, lineType=cv2.LINE_AA)
                cv2.putText(frame, txt, (int(ct[0]), int(ct[1])), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                            c, thickness=2, lineType=cv2.LINE_AA)

            cv2.putText(frame, str(frame_index), (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, 
                        (0, 0, 0), thickness=3, lineType=cv2.LINE_AA)
            cv2.putText(frame, str(frame_index), (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, 
                        (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)

            out.write(frame)
            cv2.imshow('image',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cv2.destroyAllWindows()
        out.release()
    clips = None

==================== game1 Clip1 ====================
==================== game1 Clip10 ====================
==================== game1 Clip11 ====================
==================== game1 Clip12 ====================
==================== game1 Clip13 ====================
==================== game1 Clip2 ====================
==================== game1 Clip3 ====================
==================== game1 Clip4 ====================
==================== game1 Clip5 ====================
==================== game1 Clip6 ====================
==================== game1 Clip7 ====================
==================== game1 Clip8 ====================
==================== game1 Clip9 ====================
==================== game10 Clip1 ====================
==================== game10 Clip10 ====================
==================== game10 Clip11 ====================
==================== game10 Clip12 ====================
==================== game10 Clip2 ====================
================

In [316]:
print(game, clip, movie_save_path)
Video(movie_save_path, width=640)

game9 Clip9 ../output/player/movie_process/game9/Clip9.mp4


In [308]:
mot_pid_df.to_csv(mot_process_save_dir, game, '{}.csv'.format(clip), index=False)

,frame_index,xmin,ymin,xmax,ymax,width,height,tracking_id,confidence,person_name
0,1,442,220,521,490,79,270,6,0.534033,Unknown_WhiteWhite
1,2,435,221,520,489,84,267,6,0.501841,Unknown_WhiteWhite
2,3,425,223,518,487,93,263,6,0.599908,Unknown_WhiteWhite
3,4,409,226,518,485,108,258,6,0.554233,Unknown_WhiteWhite
4,5,396,229,518,484,121,255,6,0.408226,Unknown_WhiteWhite
...,...,...,...,...,...,...,...,...,...,...
197,97,640,91,675,155,35,64,7,0.667760,Unknown_WhiteBlack
198,98,637,92,672,154,35,62,7,0.730736,Unknown_WhiteBlack
199,99,635,93,671,154,35,61,7,0.722021,Unknown_WhiteBlack
200,100,633,93,668,153,35,60,7,0.691519,Unknown_WhiteBlack
